In [1]:
# Initialise the settings
from prompting import settings
settings.settings = settings.Settings(mode="validator")
settings = settings.settings 

2024-09-06 07:49:14.886 | INFO     | prompting:<module>:17 - Project version: 2.7.2


2024-09-06 07:49:17.360 | INFO     | prompting.utils.config:config:34 - RUNNING WITH ARGS: netuid=None wallet.name=None wallet.hotkey=None subtensor.network=None axon.port=None
2024-09-06 07:49:17.362 | INFO     | prompting.utils.config:config:35 - UNKNOWN ARGS: ['--f=/root/.local/share/jupyter/runtime/kernel-v2-904744jocVEzQNB1yR.json']
2024-09-06 07:49:17.364 | INFO     | prompting.settings:<module>:12 - Config: 
netuid: null
wallet:
  name: null
  hotkey: null
subtensor:
  network: null
axon:
  port: null
no_prompt: false
config: null
strict: false
no_version_checking: false

2024-09-06 07:49:17.504 | INFO     | prompting.utils.config:config:34 - RUNNING WITH ARGS: netuid=None wallet.name=None wallet.hotkey=None subtensor.network=None axon.port=None
2024-09-06 07:49:17.506 | INFO     | prompting.utils.config:config:35 - UNKNOWN ARGS: ['--f=/root/.local/share/jupyter/runtime/kernel-v2-904744jocVEzQNB1yR.json']
2024-09-06 07:49:17.507 | INFO     | prompting.settings:load_env:150 - Ins

## LLM Pipeline

In [2]:
# Initialise the LLM we use on the validator
from prompting.llms.vllm_llm import vLLMPipeline
pipeline = vLLMPipeline(llm_model_id="casperhansen/llama-3-8b-instruct-awq", llm_max_allowed_memory_in_gb=20, device="CUDA", quantization=False, llm_max_model_len=2048)

/workspace/prompting/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-09-06 07:49:19,667	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-09-06 07:49:19.917 | INFO     | prompting.llms.utils:calculate_single_gpu_requirements:20 - Available free memory: 84.53 GB
2024-09-06 07:49:19.919 | INFO     | prompting.llms.utils:calculate_single_gpu_requirements:21 - Total gpu memory 84.97 GB
2024-09-06 07:49:19.920 | INFO     | prompting.llms.utils:calculate_single_gpu_requirements:31 - 24.0% of the GPU memory will be utilized for loading the model to device "CUDA".
2024-09-06 07:49:19.920 | INFO     | prompting.llms.vllm_llm:load_vllm_pipeline:43 - Loading VLLM pipeline with model_id casperhansen/llama-3-8b-instruct-awq: Max. VRAM: 0.24; GPUs: 1


INFO 09-06 07:49:19 awq_marlin.py:77] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 09-06 07:49:19 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='casperhansen/llama-3-8b-instruct-awq', speculative_config=None, tokenizer='casperhansen/llama-3-8b-instruct-awq', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=casperhansen/llama-3-8b-instruct-awq, use_v2_block_manager=False, ena

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  5.81it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.77it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.96it/s]



INFO 09-06 07:49:22 model_runner.py:692] Loading model weights took 5.3453 GB
INFO 09-06 07:49:23 gpu_executor.py:102] # GPU blocks: 6329, # CPU blocks: 2048


In [3]:
# as you can see, "pipeline" is an object that simply wraps around the LLM and is callable
pipeline("What's 1+2?").split("\n")[0]

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.24s/it, est. speed input: 2.17 toks/s, output: 79.19 toks/s]


' * 1+2 is 3.'

## Dataset

Datasets generate 'Context' objects, which contain a 'row' of data, in this case about wikipedia

In [4]:
from prompting.datasets.wiki import WikiDataset
dataset = WikiDataset()
context = dataset.random()
context

Context(title='Eightmile Island', topic='Contents', subtopic='Eightmile Island is a forested island on the Ohio River in Mason County, West Virginia. The island is located directly across the river from the village of Cheshire, Ohio and American Electric Power Company power plant facilities there.\n', content='Eightmile Island is a forested island on the Ohio River in Mason County, West Virginia. The island is located directly across the river from the village of Cheshire, Ohio and American Electric Power Company power plant facilities there.\n', internal_links=[], external_links=['American Electric Power Company', 'Cheshire, Ohio', 'Island', 'Ohio', 'Ohio River', 'Power plant', 'Mason County, West Virginia', 'West Virginia', 'List of islands of West Virginia', 'Geographic coordinate system'], source='Wikipedia', tags=['Coordinates on Wikidata', 'Infobox mapframe without OSM relation ID on Wikidata', 'Islands of the Ohio River', 'Landforms of Mason County, West Virginia', 'Metro Valley

## Tasks

Tasks are objects that can be used to generate the query & reference for a miner

### Initialise with past data

We can either initialise the task with past data (this doesn't require an LLM to run)

In [5]:
from prompting.tasks.summarization import SummarizationTask, SummarizationRewardConfig
SummarizationTask.generate_query_reference(llm_pipeline=pipeline, context=context)

2024-09-06 07:49:34.651 | INFO     | prompting.tasks.base_task:generate_query:59 - 🤖 Generating query...
Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.27s/it, est. speed input: 19.61 toks/s, output: 78.46 toks/s]
2024-09-06 07:49:37.930 | INFO     | prompting.llms.vllm_llm:_forward:194 - vLLM_LLM generated the following output:


To generate a request about Eight Mile Island, I'd like to request a detailed summary of the topic. Here's a breakdown of what I'd like to know:

**Background Information**: 
Please provide a brief overview of Eight Mile Island, including its geographical location and any notable features. For instance, is it an island in a specific country or body of water? Are there any notable landmarks, such as lighthouses or beaches?

**Historical Significance**: 
I'd like to know about any significant historical events or milestones that have occurred on or around Eight Mile Island. This could include notable shipwrecks, pirate sightings, or historical events 

("\n\nTo generate a request about Eight Mile Island, I'd like to request a detailed summary of the topic. Here's a breakdown of what I'd like to know:\n\n**Background Information**: \nPlease provide a brief overview of Eight Mile Island, including its geographical location and any notable features. For instance, is it an island in a specific country or body of water? Are there any notable landmarks, such as lighthouses or beaches?\n\n**Historical Significance**: \nI'd like to know about any significant historical events or milestones that have occurred on or around Eight Mile Island. This could include notable shipwrecks, pirate sightings, or historical events that have impacted the island's development.\n\n**Environmental Characteristics**: \nPlease describe the natural environment of Eight Mile Island, including its vegetation, wildlife, and climate. Are there any unique or endangered species found on the island? How does the island's ecosystem interact with its surrounding environme

## Miner Responses

Now let's say we have a few miners giving us responses

In [6]:
import numpy as np
from prompting.base.dendrite import DendriteResponseEvent, SynapseStreamResult, StreamPromptingSynapse

miner_response_1 = SynapseStreamResult(synapse=StreamPromptingSynapse(completion="4", roles=["user"], messages=["What's 1+2?"]))
miner_response_2 = SynapseStreamResult(synapse=StreamPromptingSynapse(completion="3", roles=["assistant"], messages=["What's 1+2?"]))


# the synapses from all miners get collected into the DenriteResponseEvent
dendrite_response = DendriteResponseEvent(stream_results=[miner_response_1, miner_response_2], uids=np.array([1, 2]), timeout=10)

# Scoring

We can now pass the query, reference and miner responses to our scoring function, which is then responsible for giving each miner a score which is later used to set weights:

In [7]:
from prompting.tasks.summarization import SummarizationRewardConfig

reward_events, penality_events, rewards = SummarizationRewardConfig.apply(challenge="What's 1+2?", reference="1+2 is equal to 3", response_event=dendrite_response)
rewards

array([0.17160807, 0.39458477])

# Other tests/examples on different tasks

In [8]:
from prompting.tasks.qa import QuestionAnsweringTask, QARewardConfig
qa = QuestionAnsweringTask(context=context.model_dump(), llm_pipeline=pipeline, reward_config=SummarizationRewardConfig())

In [9]:
# Used to obtain the query (which is a question about the context)
QUERY_PROMPT_TEMPLATE = """\
Ask a specific question about the following context:

#Context:
{context}

You must ask a question that can be answered by the context.
"""

In [10]:
query_prompt = QUERY_PROMPT_TEMPLATE.format(context=context.content)
query = qa.generate_query(llm_pipeline=pipeline, message=query_prompt)
query


2024-09-06 07:49:38.765 | INFO     | prompting.tasks.base_task:generate_query:59 - 🤖 Generating query...
Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.97it/s, est. speed input: 439.14 toks/s, output: 50.78 toks/s]
2024-09-06 07:49:39.108 | INFO     | prompting.llms.vllm_llm:_forward:194 - vLLM_LLM generated the following output:


What is the location of Eightmile Island in relation to the Ohio River?


'\n\nWhat is the location of Eightmile Island in relation to the Ohio River?'

In [11]:
# Used to obtain reference answer
REFERENCE_PROMPT_TEMPLATE = """\
Answer the question you will receive in detail, utilizing the following context.

#Context:
{context}

# Question:
{question}
"""

In [12]:
reference_prompt = REFERENCE_PROMPT_TEMPLATE.format(context=context.content, question=query)
reference = qa.generate_reference(llm_pipeline=pipeline, messages=[reference_prompt])
reference

2024-09-06 07:49:39.124 | INFO     | prompting.tasks.base_task:generate_reference:46 - 🤖 Generating reference...
Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.09s/it, est. speed input: 73.47 toks/s, output: 77.31 toks/s]
2024-09-06 07:49:41.216 | INFO     | prompting.llms.vllm_llm:_forward:194 - vLLM_LLM generated the following output:


Based on the provided context, Eightmile Island is located on the Ohio River in Mason County, West Virginia. This means that the island is situated directly in the Ohio River, with a unique characteristic that it is located directly across the river from the village of Cheshire, Ohio, and the American Electric Power Company power plant facilities there.

To be more specific, Eightmile Island is not located on the banks of the Ohio River, but rather it is a forested island that is already situated in the river itself. This makes it a distinctive and isolated geographical feature, with the river flowing around it. The proximity to the village 

"\n\nBased on the provided context, Eightmile Island is located on the Ohio River in Mason County, West Virginia. This means that the island is situated directly in the Ohio River, with a unique characteristic that it is located directly across the river from the village of Cheshire, Ohio, and the American Electric Power Company power plant facilities there.\n\nTo be more specific, Eightmile Island is not located on the banks of the Ohio River, but rather it is a forested island that is already situated in the river itself. This makes it a distinctive and isolated geographical feature, with the river flowing around it. The proximity to the village of Cheshire, Ohio, and the power plant facilities also highlights the island's strategic location, which is likely to have played a role in its historical significance or usage."